## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [21]:
import numpy as np
import pandas as pd
import pickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## CARGAR LOS DATOS

### Ruta del proyecto

In [22]:
ruta_proyecto = 'C:/Users/Marius/EstructuraDirectorio/03_MACHINE_LEARNING/07_CASOS/03_RIESGOS'

### Nombre del fichero de datos

In [23]:
nombre_fichero_datos = 'prestamos.csv'

### Cargar los datos

In [24]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col=0)

### Seleccionar solo las variables finales

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [25]:
variables_finales = ['ingresos_verificados',
                     'vivienda',
                     'finalidad',
                     'num_cuotas',
                     'antigüedad_empleo',
                     'rating',
                     'ingresos',
                     'dti',
                     'num_lineas_credito',
                     'porc_uso_revolving',
                     'principal',
                     'tipo_interes',
                     'imp_cuota',
                     'num_derogatorios',
                     'estado',
                     'imp_amortizado',
                     'imp_recuperado'
                  ]

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Eliminar registros

#### Por atípicos

In [26]:
a_eliminar = df.loc[df.ingresos > 300000].index.values


In [27]:
df = df[~df.index.isin(a_eliminar)]

### Seleccionar variables

Quedarse solo con las de la lista.

In [28]:
df = df[variables_finales]
df

,ingresos_verificados,vivienda,finalidad,num_cuotas,antigüedad_empleo,rating,ingresos,dti,num_lineas_credito,porc_uso_revolving,principal,tipo_interes,imp_cuota,num_derogatorios,estado,imp_amortizado,imp_recuperado
id_cliente,,,,,,,,,,,,,,,,,
137387967,Source Verified,MORTGAGE,debt_consolidation,36 months,3 years,A,54000.0,19.31,10.0,45.2,15000.0,7.21,464.60,0.0,Current,2669.06,0.00
4798121,Not Verified,RENT,debt_consolidation,36 months,10+ years,D,65000.0,25.40,15.0,86.2,10000.0,17.77,360.38,0.0,Charged Off,6362.96,0.00
46641215,Verified,RENT,debt_consolidation,36 months,5 years,A,135000.0,14.68,19.0,16.3,24000.0,6.39,734.38,0.0,Fully Paid,24000.00,0.00
87998444,Source Verified,MORTGAGE,credit_card,60 months,9 years,B,188000.0,11.69,15.0,3.3,27000.0,8.99,560.35,0.0,Current,12443.00,0.00
132883631,Source Verified,MORTGAGE,debt_consolidation,36 months,7 years,B,125000.0,9.00,6.0,69.9,22000.0,10.90,719.22,0.0,Fully Paid,22000.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51876926,Not Verified,MORTGAGE,debt_consolidation,36 months,10+ years,C,42000.0,20.85,9.0,81.6,8000.0,12.29,266.83,0.0,Fully Paid,8000.00,0.00
121031962,Verified,MORTGAGE,other,36 months,6 years,B,111697.0,16.63,10.0,26.3,10000.0,9.44,320.05,1.0,Current,4388.51,0.00
135641397,Source Verified,MORTGAGE,small_business,36 months,10+ years,D,285000.0,6.02,9.0,21.1,30000.0,17.47,1076.62,0.0,Current,5387.53,0.00


## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [29]:
#funcion unica que servira para los 3 modelos por igual
def calidad_datos(temp):
    
    #esta variable categorica la habiamos imputado por valor desonocido
    temp['antigüedad_empleo'] = temp['antigüedad_empleo'].fillna('desconocido')
    
    #todas las numericas las imputamos por 0
    temp.update(temp.select_dtypes('number').fillna(value = 0))
    
    return(temp)

### Instanciar creación de variables

Dado que la creación de variables es diferente para los 3 modelos necesitamos construir 3 funciones.

#### Crear las funciones

In [30]:
def creacion_variables_pd(df):
    
    temp = df.copy()
    
    temp['target_pd'] = np.where(temp.estado.isin(['Charged Off','Does not meet the credit policy. Status:Charged Off','Default']), 1, 0)
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    #las eliminamos porque no tendremos esta informacion a la hora de hacer esta evaluacion de riesgos 
    #para saber si hacer el prestamo o no
    
    #la variable estado la hemos utilizado para hacer nuestra target, entonces ya no puede ser usada en nuestro df
    #porque sino seria usada como una predictora y no podemos usarla como predictora porque es la variable madre de la var target
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

In [31]:
def creacion_variables_ead(df):
    
    temp = df.copy()
    
    temp['pendiente'] = temp.principal - temp.imp_amortizado
    
    temp['target_ead'] = temp.pendiente / temp.principal
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    #ppendiente fue creada por nosotros para poder crear la target por lo tanto eliminamos
    #y las demas, son variables que no dispondremos de esa informacion en el momento de hacer la evaluacion de riesgos
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado','pendiente'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

In [32]:
def creacion_variables_lgd(df):
    
    temp = df.copy()
    
    temp['pendiente'] = temp.principal - temp.imp_amortizado
    
    temp['target_lgd'] = 1 - (temp.imp_recuperado / temp.pendiente)
    
    temp['target_lgd'].fillna(0,inplace=True)
    
    temp.vivienda = temp.vivienda.replace(['ANY','NONE','OTHER'],'MORTGAGE')
    
    temp.finalidad = temp.finalidad.replace(['wedding','educational','renewable_energy'],'otros')
    
    #Eliminamos las variables que ya no usaremos
    temp.drop(columns = ['estado','imp_amortizado','imp_recuperado','pendiente'],inplace = True)
    
    #Separamos entre predictoras y target
    temp_x = temp.iloc[:,:-1]
    temp_y = temp.iloc[:,-1]
    
    return(temp_x,temp_y)

#### Crear los dataframes de X e y

In [33]:
#Python ejecuta primero la funcion de dentro (calidad de datos) y despues el resultado que esta dentro le pasa como parametro
#de la siguiente funcion
x_pd, y_pd = creacion_variables_pd(calidad_datos(df))

x_ead, y_ead = creacion_variables_ead(calidad_datos(df))

x_lgd, y_lgd = creacion_variables_lgd(calidad_datos(df))

### Instanciar transformación de variables

In [34]:
#ONE HOT ENCODING
var_ohe = [ 'ingresos_verificados', 'vivienda','finalidad','num_cuotas']
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')


#ORDINAL ENCODING
var_oe = ['antigüedad_empleo','rating']

orden_antigüedad_empleo = ['desconocido','< 1 year','1 year','2 years','3 years','4 years',
                           '5 years','6 years','7 years','8 years','9 years','10+ years']

orden_rating = ['A','B','C','D','E','F','G']

oe = OrdinalEncoder(categories = [orden_antigüedad_empleo,orden_rating],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 12)

#BINNING
var_bin = ['num_derogatorios']
bina = Binarizer(threshold=0)


#MIN-MAX SCALING
var_mms = ['ingresos','dti','num_lineas_credito','porc_uso_revolving',
            'principal','tipo_interes','imp_cuota']
mms = MinMaxScaler()


### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [35]:
#Column transformer permite aplicar variables diferentes a procesos diferentes

ct = make_column_transformer(
    (ohe, var_ohe),
    (oe, var_oe),
    (bina, var_bin),
    (mms, var_mms),
    remainder='passthrough')

### Instanciar los modelos

#### Instanciar los algoritmos

In [36]:
modelo_pd = LogisticRegression(solver = 'saga', n_jobs=-1, C = 0.25, penalty = 'l1')

modelo_ead = HistGradientBoostingRegressor(learning_rate = 0.1,
                                          max_iter = 200,
                                          max_depth = 10,
                                          min_samples_leaf = 100,
                                          scoring = 'neg_mean_absolute_percentage_error',
                                          l2_regularization = 0.75)

modelo_lgd = HistGradientBoostingRegressor(learning_rate = 0.1,
                                          max_iter = 200,
                                          max_depth = 20,
                                          min_samples_leaf = 100,
                                          scoring = 'neg_mean_absolute_percentage_error',
                                          l2_regularization = 0)

#### Crear los pipes finales de entrenamiento

In [37]:
pipe_entrenamiento_pd = make_pipeline(ct,modelo_pd)

pipe_entrenamiento_ead = make_pipeline(ct,modelo_ead)

pipe_entrenamiento_lgd = make_pipeline(ct,modelo_lgd)

#Ahora ya tenemos creado y definido todo el trabajo que tenemos que hacer con pandas mediante funciones y 
#todo el trabajo que tenemos que hacer con scikit-learn, mediante pipes

#### Guardar el pipe final de entrenamiento

En esta parte vamos a guardar toda esa definicion, ya que todavia no hemos entrenado estos modelos, lo que estamos guardando son los objetos de entrenamiento de scikit-learn, que contienen preprocesamiento de scikit-learn como modelización.
Estos 3 pipes de entrenamiento son los que vamos a recuperar en el codigo de reentrenamiento, cada vez que queramos hacer un reentrenamiento de los modelos vamos a utilizar la definicion de ese entrenamiento pero vamos a reentrenarlo.

In [38]:
ruta_pipe_entrenamiento_pd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_pd.pickle'

with open(ruta_pipe_entrenamiento_pd, mode='wb') as file:
    pickle.dump(pipe_entrenamiento_pd, file)

In [39]:
ruta_pipe_entrenamiento_ead = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_ead.pickle'

with open(ruta_pipe_entrenamiento_ead, mode='wb') as file:
    pickle.dump(pipe_entrenamiento_ead, file)

In [40]:
ruta_pipe_entrenamiento_lgd = ruta_proyecto + '/04_Modelos/pipe_entrenamiento_lgd.pickle'

with open(ruta_pipe_entrenamiento_lgd, mode='wb') as file:
    pickle.dump(pipe_entrenamiento_lgd, file)

#### Entrenar los pipes

In [41]:
#Entrenamos para nuestro codigo de ejecucion y despues guardaremos los objetos entrenados
#estos objetos no seria necesario reentrenarlos
pipe_ejecucion_pd = pipe_entrenamiento_pd.fit(x_pd,y_pd)
pipe_ejecucion_ead = pipe_entrenamiento_ead.fit(x_ead,y_ead)
pipe_ejecucion_lgd = pipe_entrenamiento_lgd.fit(x_lgd,y_lgd)

C:\Users\Marius\miniconda3\envs\riskscoring\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Marius\miniconda3\envs\riskscoring\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Marius\miniconda3\envs\riskscoring\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## GUARDAR EL PIPE

### Guardar el pipe final de ejecución

In [42]:
#Tanto los scripts de reentrenamiento como de ejecucion son material de producción

ruta_pipe_ejecucion_pd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_pd.pickle'

with open(ruta_pipe_ejecucion_pd, mode='wb') as file:
    pickle.dump(pipe_ejecucion_pd, file)

In [43]:
ruta_pipe_ejecucion_ead = ruta_proyecto + '/04_Modelos/pipe_ejecucion_ead.pickle'

with open(ruta_pipe_ejecucion_ead, mode='wb') as file:
    pickle.dump(pipe_ejecucion_ead, file)

In [44]:
ruta_pipe_ejecucion_lgd = ruta_proyecto + '/04_Modelos/pipe_ejecucion_lgd.pickle'

with open(ruta_pipe_ejecucion_lgd, mode='wb') as file:
    pickle.dump(pipe_ejecucion_lgd, file)